## 項目1 使用するPythonのバージョンを知っておく

In [10]:
!python --version

Python 3.8.5


In [11]:
import sys
print(sys.version_info)
print(sys.version)

sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


## 項目2 PEP8スタイルガイドに従う

以下のスタイルガイドがある。

* 空白
* 名前付け
* 式と文
* インポート

## 項目3 bytesとstrの違い

In [15]:
a = b'h\x65llo'
print(list(a))
print(a)

[104, 101, 108, 108, 111]
b'hello'


In [18]:
a = 'a\u0300 propos'
print(list(a))
print(a)

['a', '̀', ' ', 'p', 'r', 'o', 'p', 'o', 's']
à propos


Unicodeサンドイッチ（strとbytesの違いで処理に誤りがおきるので変換する）

In [21]:
def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode('utf-8')
    else:
        value = bytes_or_str
    return value # str instance

print(repr(to_str(b'foo')))
print(repr(to_str('bar')))

'foo'
'bar'


In [23]:
def to_bytes(bytes_or_str):
    if isinstance(bytes_or_str, str):
        value = bytes_or_str.encode('utf-8')
    else:
        value = bytes_or_str
    return value # bytes instance

print(repr(to_bytes(b'foo')))
print(repr(to_bytes('bar')))

b'foo'
b'bar'


その他の注意

* strとbytesでは同じように扱えない
  * str、bytes同士は+演算子で結合できる、比較、同値チェックができる
    * str、bytes間ではできないので注意

In [2]:
print(b'one', b'two')
print('one' + 'two')

b'one' b'two'
onetwo


In [3]:
b'one' + 'two'

TypeError: can't concat str to bytes

In [4]:
'one' + b'two'

TypeError: can only concatenate str (not "bytes") to str

In [6]:
assert b'red' > b'blue'
assert 'red' > 'blue'

In [7]:
assert 'red' > b'blue'

TypeError: '>' not supported between instances of 'str' and 'bytes'

In [9]:
assert b'blue' < 'red'

TypeError: '<' not supported between instances of 'bytes' and 'str'

In [10]:
print(b'foo' == 'foo')

False


In [11]:
print(b'red %s' % b'blue')
print('red %s' % 'blue')

b'red blue'
red blue


In [12]:
print(b'red %s' % 'blue')

TypeError: %b requires a bytes-like object, or an object that implements __bytes__, not 'str'

In [14]:
# bytesの__repr__によって b'blue' がそのまま表示される
print('red %s' % b'blue')

red b'blue'


ファイル書き込み（バイナリ、文字列の出力）

In [27]:
with open('chapter01/random.bin', 'w') as f:
    f.write(b'\xf1\xf2\xf3\xf4\xf5')

TypeError: write() argument must be str, not bytes

In [28]:
with open('chapter01/random.bin', 'wb') as f:
    f.write(b'\xf1\xf2\xf3\xf4\xf5')

In [29]:
with open('chapter01/random.bin', 'r') as f:
    data = f.read()

UnicodeDecodeError: 'cp932' codec can't decode byte 0xf5 in position 4: incomplete multibyte sequence

In [30]:
with open('chapter01/random.bin', 'rb') as f:
    data = f.read()
assert data == b'\xf1\xf2\xf3\xf4\xf5'

In [33]:
with open('chapter01/random.bin', 'r', encoding='cp1252') as f:
    data = f.read()

assert data == 'ñòóôõ'

## 項目4 Cスタイルフォーマット文字列とstr.formatは使わずf文字列で埋め込む

### %による方法

* Cスタイルに慣れていればそのまま使える
* フォーマットの型を揃える必要がある（順序が変わるとフォーマット側も変える必要がある）

In [34]:
a = 0b10111011
b = 0xc5f
print('Binary is %d, hex is %d' % (a,b))

Binary is 187, hex is 3167


In [35]:
key = 'my_var'
value = 1.234
formatted = '%-10s = %.2f' % (key, value)
print(formatted)

my_var     = 1.23


In [37]:
key = 'my_var'
value = 1.234
formatted = '%-10s = %.2f' % (value, key)
print(formatted)

TypeError: must be real number, not str

In [39]:
key = 'my_var'
value = 1.234
formatted = '%.2f = %-10s' % (key, value)
print(formatted)

TypeError: must be real number, not str